#cupy

In [79]:
import cupy as cp
import numpy as np

In [91]:
array_cpu = np.random.randint(0,255, size=(4000,4000)) # cpu array creation in ram

print("The array:", array_cpu)
print("\nMemery sizr:", array_cpu.nbytes/1e6, "MB")

The array: [[109  43 177 ... 159 250  73]
 [229 221 201 ... 194  37  32]
 [ 10  68  20 ... 239   9 224]
 ...
 [ 68 118  31 ...  62 168   1]
 [182   7 141 ... 121  81  72]
 [143  89 233 ... 183  61  52]]

Memery sizr: 64.0 MB


In [92]:
%%time
#sending the array to the gpu vram

array_gpu = cp.asarray(array_cpu)

print("The gpu array:",array_gpu)

#we have sent the array to the gpu

The gpu array: [[109  43 177 ... 159 250  73]
 [229 221 201 ... 194  37  32]
 [ 10  68  20 ... 239   9 224]
 ...
 [ 68 118  31 ...  62 168   1]
 [182   7 141 ... 121  81  72]
 [143  89 233 ... 183  61  52]]
CPU times: total: 31.2 ms
Wall time: 109 ms


first fourier transform

In [100]:
%%time
from scipy import fft

fft.fftn(array_cpu) # cpu and ram involve

CPU times: total: 78.1 ms
Wall time: 349 ms


array([[ 2.03202444e+09     -0.j        ,
         2.07410168e+05 -29115.30873688j,
        -2.27306906e+05  -2795.3595448j , ...,
         1.16844376e+05 +72008.68798133j,
        -2.27306906e+05  +2795.3595448j ,
         2.07410168e+05 +29115.30873688j],
       [ 5.53361005e+04-393086.24518482j,
        -8.96043372e+04+234214.04014656j,
         8.38105771e+04-247236.80299501j, ...,
        -2.24061640e+05-266174.81955591j,
        -2.02967285e+05-114947.41075849j,
        -2.13543548e+05+320596.7673818j ],
       [-2.17245789e+05-213377.45623747j,
         1.72381607e+05+192697.67176335j,
        -5.84336210e+04+283666.30670998j, ...,
         1.42165830e+05 +41149.86496527j,
         2.63543326e+05-109229.44643383j,
         8.78567990e+04-100444.74197046j],
       ...,
       [ 3.13694218e+05-219727.06071734j,
        -1.03644190e+05-153179.31934279j,
         1.08770092e+05+269455.54156163j, ...,
        -2.13458114e+05 -22900.30689727j,
        -2.74357898e+05+134566.31406861j,

In [104]:
%%time
from cupyx.scipy import fft as fft_gpu

fft_gpu.fftn(array_gpu) # cpu and

CPU times: total: 0 ns
Wall time: 998 µs


array([[ 2.03202444e+09     +0.j        ,
         2.07410168e+05 -29115.30873688j,
        -2.27306906e+05  -2795.3595448j , ...,
         1.16844376e+05 +72008.68798133j,
        -2.27306906e+05  +2795.3595448j ,
         2.07410168e+05 +29115.30873688j],
       [ 5.53361005e+04-393086.24518482j,
        -8.96043372e+04+234214.04014656j,
         8.38105771e+04-247236.80299501j, ...,
        -2.24061640e+05-266174.81955591j,
        -2.02967285e+05-114947.41075849j,
        -2.13543548e+05+320596.7673818j ],
       [-2.17245789e+05-213377.45623747j,
         1.72381607e+05+192697.67176335j,
        -5.84336210e+04+283666.30670998j, ...,
         1.42165830e+05 +41149.86496527j,
         2.63543326e+05-109229.44643383j,
         8.78567990e+04-100444.74197046j],
       ...,
       [ 3.13694218e+05-219727.06071734j,
        -1.03644190e+05-153179.31934279j,
         1.08770092e+05+269455.54156163j, ...,
        -2.13458114e+05 -22900.30689727j,
        -2.74357898e+05+134566.31406861j,

In [111]:
#communicate host cpu and gpu


fft_cpu = fft.fftn(array_cpu)
fft_send_back = cp.asnumpy(fft_gpu.fftn(array_gpu))   #sending back to the cpu as numpy array

np.allclose(fft_cpu,fft_send_back)    #comparing the array of np and cp


True

In [114]:
import cupy as cp
import numpy as np
#at a glance communicating cpu and gpu

#step - 1: making a np array
array_cpu = np.random.randint(0,255, size=(4000,4000)) # cpu array creation in ram

#step - 2: sending the array to the gpu
array_gpu = cp.asarray(array_cpu)


#step - 3: makig fft with np array in gpu

from cupyx.scipy import fft as fft_gpu
fft_in_gpu = fft_gpu.fftn(array_gpu)


#step - : sending back to the cpu as numpy array
fft_send_back = cp.asnumpy(fft_in_gpu)


In [ ]:
type(fft_send_back)

numpy.ndarray

In [115]:
#testing
fft_cpu = fft.fftn(array_cpu) #fft on cpu
np.allclose(fft_cpu,fft_send_back) 

True

In [120]:
#creating array dorect on gpu

array_gpu_direct = cp.random.randint(0,255, size=(4000,4000))
print(array_gpu_direct)

#convert it to numpy
array_numpy = cp.asnumpy(array_gpu_direct)

print(type(array_numpy))

[[220 124  75 ...  64 193 183]
 [225 174 167 ... 196  22 231]
 [ 58 231  38 ... 222 192 124]
 ...
 [ 99  47   9 ... 141 108 186]
 [162 178  31 ... 117 146 254]
 [180 125  49 ...  77 243 154]]
<class 'numpy.ndarray'>


#numba.cuda

In [70]:
# Importing NumPy
import numpy as np  

# Using numpy.random.rand() to generate an array of random numbers of size (2, 2)
random_array1 = np.random.rand(N)
radnom_array2 = np.random.rand(N)


In [71]:
ans =[]
for i in range(random_array1.size):
    ans.append(random_array1[i]*radnom_array2[i]) 
    

In [55]:
print(ans)

[0.049600174945875306, 0.04836575691397883, 0.0945811000522485, 0.19367070304946155, 0.2378207232994692, 0.015235294144712138, 0.04440892620195568, 0.017134004732664118, 0.056112696600885296, 0.02405702963315841, 0.1056482713860908, 0.5803322458977875, 0.3410060045419952, 0.8611443092637028, 0.22747057129413334, 0.10556723627977306, 0.11142799645355056, 0.31009838020032765, 0.014251867605170297, 0.09864654439669371, 0.2625767947646237, 0.04259395296053167, 0.26944149160866504, 0.6945953187058631, 0.1636652707965866, 0.5334470112128289, 0.6621028441820781, 0.2600436013013477, 0.3886417266075598, 0.5023589670127603]


In [37]:
print("Testing git")

Testing git
